In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Thesis_code

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from utils import file_helper, feature_extraction

# Data Pipeline

In [ ]:
train_X = np.load('Dataset/split_data/Multi-Class Split/X_train.npy')
train_y = np.load('Dataset/split_data/Multi-Class Split/y_train.npy')
val_X = np.load('Dataset/split_data/Multi-Class Split/X_val.npy')
val_y = np.load('Dataset/split_data/Multi-Class Split/y_val.npy')
test_X = np.load('Dataset/split_data/Multi-Class Split/X_test.npy')
test_y = np.load('Dataset/split_data/Multi-Class Split/y_test.npy')

# Note: 
# Class     Label     One-hot Encoding
# HUMAN       0     [1, 0, 0, 0 ,0]
# BICYCLE     1     [0, 1, 0, 0, 0]
# PILLAR      2     [0, 0, 1, 0, 0]
# WALL        3     [0, 0, 0, 1, 0]
# CAR         4     [0, 0, 0, 0, 1]

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y)).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((val_X, val_y)).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_y)).batch(BATCH_SIZE)


# CNN Model: 

## Build process: 

In [ ]:
tf.keras.backend.clear_session()
METRICS = [ 
      tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),

]
model = tf.keras.models.Sequential([
    #1st
    tf.keras.layers.Conv1D(54,kernel_size = 5, input_shape = (1,5704), padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.1),
    #2nd
    tf.keras.layers.Conv1D(128,5, padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.1),
    #3rd
    tf.keras.layers.Conv1D(256,5, padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.1),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation= 'relu'), 
    tf.keras.layers.Dense(5, activation = 'softmax') 
])



# Optimizer

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam

optimizer_R = tf.keras.optimizers.RMSprop(learning_rate = 1e-2,momentum=0.9)
optimizer_A = tf.keras.optimizers.Adam(learning_rate = 1e-2)

model.compile(optimizer = optimizer_A , loss = tf.keras.losses.CategoricalCrossentropy(), metrics =[METRICS])


In [ ]:
model.summary()

## Callbacks:

In [ ]:
import os 
from tensorflow.keras.callbacks import EarlyStopping

#Early stop to avoid overfitting
early_stop = EarlyStopping(patience = 5, verbose = 1)

#Provides unique names for checkpoints and adjust the checkpointing frequency
checkpoint_path = "CNN_Multi_CNNonly_1/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create a call back that save the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1,
                                                 period = 10)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose= 1, min_lr=1e-9)

# Train the Model

In [ ]:
model_train = model.fit(train_dataset,
                        epochs = 100,
                        validation_data = val_dataset, 
                        callbacks = [reduce_lr, cp_callback]
                        )#callbacks = [cp_callback, reduce_lr]

In [ ]:
model.save("CNN_Multiclass.h5")

# Plot Accuracy vs Loss

In [ ]:
plt.figure(figsize =(20,7))

plt.subplot(1,2,1)
plt.plot(model_train.epoch, model_train.history["loss"], label="Train")
plt.plot(model_train.epoch, model_train.history["val_loss"], label="Valid")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(model_train.epoch, model_train.history["categorical_accuracy"], label="Train")
plt.plot(model_train.epoch, model_train.history["val_categorical_accuracy"], label="Valid")
plt.ylim([0,1.0])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Test and evaluate the model

In [ ]:
test_eval = model.predict(test_X, batch_size = 64)
model.evaluate(test_dataset, batch_size= 64, verbose = 1)

# Plot classification report

In [ ]:
from sklearn.metrics import classification_report
pred = np.round(test_eval, 0)
label = ["HUMAN","BICYCLE","PILLAR", "WALL","CAR"]
target = ["Class {}".format(i) for i in range(5)]
classification_metrics = classification_report(test_y.argmax(1) ,pred.argmax(1), target_names = label)
print(classification_metrics)


In [ ]:
from utils.confusion_matrix_pretty_print import plot_confusion_matrix_from_data

columns = ["HUMAN","BICYCLE","PILLAR","WALL","CAR"]
annot = True
cmap = 'Oranges'
fmt = '.2f'
lw = 0.5
cbar = False
show_null_values = 2
pred_val_axis = 'y'
#size::
fz = 12
figsize = [9,9]
if(len(test_y) > 10):
  fz=9; figsize=[14,14]
plot_confusion_matrix_from_data(test_y.argmax(axis=1), pred.argmax(axis=1), columns,annot, cmap, fmt, fz, lw, cbar, figsize, show_null_values, pred_val_axis)